# Step 4: SAXS Calculation from Cluster Distributions
Calculate the small-angle X-ray scattering (SAXS) of cluster network distributions.

---

## Custom Imports
Relative import the custom classes to support the cluster network tool.

In [ ]:
# Import and run the setup script
import sys, os
import numpy as np
from typing import TypedDict, Optional, Dict

# Ensure the project root is in sys.path to locate setup_env
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import the setup helper and configure the environment
from setup_env import setup_environment

# Capture the imported classes from setup_environment
BulkVolumeParams, BulkVolume, RadiusOfGyrationCalculator, PDBEditor, TrajectoryProcessor, PDBFileHandler, Atom, ClusterNetwork, ClusterBatchAnalyzer = setup_environment()

## SAXS Cluster Batch Analysis

#### (Optional) Validate Scattering Volume Estimates

In [ ]:
# Sample Input Parameters
## PbI2 in DMSO Sample
solvent_name = 'DMS'
density_neat_solvent = 1.1    # g/cm³ for DMSO
molar_mass_solvent = 78.13    # g/mol for DMSO
molar_mass_solute = 461.0     # g/mol for PbI2

mass_percent_solute = 25.83   # 25.83% PbI2 by mass (approximately 0.8 M)
total_mass = 1.403            # grams of solution
density_solution = 1.403       # g/cm³

ionic_radii = {
    'Pb': 1.19,   # angstroms
    'I': 2.20      # angstroms
}
stoichiometry = {
    'Pb': 1,
    'I': 2
}
atomic_masses = {
    'Pb': 207.2,   # g/mol
    'I': 126.9      # g/mol
}
solute_residues = {
    'Pb': 'PBI',
    'I': 'PBI'
}

# Instantiate the BulkVolume class with the solvent name 'DMS' for DMSO
bulk_volume = BulkVolume(
    mass_percent_solute=mass_percent_solute,
    density_solution=density_solution,
    density_neat_solvent=density_neat_solvent,
    molar_mass_solvent=molar_mass_solvent,
    molar_mass_solute=molar_mass_solute,
    ionic_radii=ionic_radii,
    stoichiometry=stoichiometry,
    atomic_masses=atomic_masses,
    solute_residues=solute_residues,
    solvent_name=solvent_name,       # 3-letter uppercase string for DMSO
    total_mass=total_mass     # Optional: defaults to 100 g if not specified
)

# Perform Volume Estimation
volumes = bulk_volume.estimate_volumes()

### Initialize Cluster Batch Analyzer

#### Batch Analyzer Settings

In [3]:
%matplotlib widget

## -- DEFINE THE PDB FOLDER PATH
pdb_directory = '/Users/keithwhite/repos/MDScatter/data/PbI2_DMSO_0p8M_cr_nb/clusters_pdb_sc3p7_PbO-3'

## -- SETUP FOR FIRST COORDINATION SHELL
target_elements = ['Pb']
neighbor_elements = ['O', 'I']
distance_thresholds = {
    ('Pb', 'O'): 3.0,  # Example threshold distances in angstroms
    ('Pb', 'I'): 3.7
}

## -- FOR CHARGE DISTRIBUTION CALCULATION - USE SETUP TOOL TO SET THESE VALUES
partial_charges = {
    'Pb': (2, 6),    # Lead with a charge of 2+ and coordination number of 6
    'I': (-1, 6),    # Iodine with a charge of 1- and coordination number of 6
    'S': (-2, 6),    # Sulfur in DMSO with a neutral charge and coordination number of 2
    'O': (-2, 2),    # Oxygen in DMSO with a charge of 2- and coordination number of 2
    'C': (4, 4),     # Carbon in DMSO with a neutral charge and coordination number of 4
    'H': (1, 1)      # Hydrogen in DMSO with a neutral charge and coordination number of 1
}

## -- DEFINE THE PARAMETERS FOR BULK VOLUME ESTIMATION FROM EXPERIMENTAL MEASUREMENTS
BulkVolumeParams = {
    'mass_percent_solute': 25.83,
    'density_solution': 1.403,
    'density_neat_solvent': 1.1,
    'molar_mass_solvent': 78.13,
    'molar_mass_solute': 461.0,
    'ionic_radii': {
        'Pb': 1.19,
        'I': 2.20
    },
    'stoichiometry': {
        'Pb': 1,
        'I': 2
    },
    'atomic_masses': {
        'Pb': 207.2,
        'I': 126.9
    },
    'solute_residues': {
        'Pb': 'PBI',
        'I': 'PBI'
    },
    'solvent_name': 'DMS',
    'total_mass': 1.403
}

#### Initialization

In [ ]:
# Instantiate the ClusterBatchAnalyzer class with the necessary parameters
analyzer = ClusterBatchAnalyzer(
    pdb_directory=pdb_directory,
    target_elements=target_elements,
    neighbor_elements=neighbor_elements,
    distance_thresholds=distance_thresholds,
    charges=partial_charges,
    core_residue_names=['PBI'], 
    shell_residue_names=['DMS'],
    # volume_method='ionic_radius'  # Choose the ionic radius estimation method
    # volume_method='radius_of_gyration'  # Choose the radius of gyration method
    volume_method='bulk_volume',  # Choose the radius of gyration method,
    bulk_volume_params = BulkVolumeParams
)

#### Cluster Distribution Estimates

In [ ]:
coordination_stats_per_size = analyzer.analyze_clusters()

## Calculate SAXS

### Method 1 

In [ ]:
# Step 1: Analyze clusters to compute coordination numbers and volumes
# Specify shape_type='sphere' to use spherical approximation with radius of gyration
coordination_stats_per_size = analyzer.analyze_clusters(shape_type='sphere')
# coordination_stats_per_size = analyzer.analyze_clusters(shape_type='ellipsoid')

# Step 2: Define a range of q-values in inverse angstroms
q_values = np.linspace(0.01, 1.4, 1000)

# Step 3: Plot the total I(q) vs. q on a log-log scale
analyzer.plot_total_iq(q_values)
analyzer.save_total_iq(q_values)

# Additional Steps (Optional):
# If you want to visualize the average volume vs. cluster size using the radius of gyration,
# you can call the corresponding plot method:
analyzer.plot_average_volume_vs_cluster_size_rg()
